<a href="https://colab.research.google.com/github/karasu1982/POS_Data_Analytics/blob/main/notebook/RFM%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFM分析

RFM分析は、顧客を次の3つの軸で分析しグループ分けする方法です。

*   Recency : 最新購入日
*   Frequency：購入頻度
*   Monetary：購入金額

購入金額だけを基準にしていたデシル分析に比べて、最新購入日・購入頻度が入っていることが特徴です。また、RFM分析という順番の通り、重視すべき軸としてはR -> F -> Mの順番であるとされています。

実際の活用場面では、RFMと3軸になると理解しづらいため、2軸のマトリックスを作成して見える化するケースが多いように思いますので、ここでもそのようにアウトプットしていきたいと思います。

## 環境設定

In [1]:
%%bash
pip install duckdb-engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.1 MB/s eta 0:00:00


In [2]:
# 標準ライブラリ
import pandas as pd
import numpy as np

# データ見える化
import seaborn as sns
import matplotlib.pyplot as plt

# DuckDB
import duckdb

## データ準備

ID-POSのサンプルデータとして、下記を利用

https://www.kyoritsu-pub.co.jp/book/b10003634.html

In [ ]:
%%bash
wget https://kyoritsu-pub.sakura.ne.jp/app/file/goods_contents/2319.zip
unzip /content/2319.zip
unzip /content/DataProcess.20151001/src/data/Tafeng/Tafeng.zip

In [4]:
df = pd.read_csv("/content/Tafeng_dataset/Tafeng.csv")

# データ型を
df = df.astype({'CustID': 'object', 'ProductSubClass': 'object', 'ProductID': 'object'})

In [5]:
df.head(3)

,Time,CustID,Age,Area,ProductSubClass,ProductID,Amount,Asset,SalesPrice
0,2000-11-01 00:00:00,46855,D,E,110411,4710085120468,3,51,57
1,2000-11-01 00:00:00,539166,E,E,130315,4714981010038,2,56,48
2,2000-11-01 00:00:00,663373,F,E,110217,4710265847666,1,180,135


## データ集計

通常であれば、R = 「本日 - 最新購入日」として最新購入日からの経過日数を出すのですが、サンプルデータは、2000/11/1-2001/2/28のデータなので、仮に本日を「2001/3/1」として2001/1/1-2/28のデータについて算出してみます。

FとMは計算は通常通り、期間を上記に合わせて算出します。


In [35]:
import duckdb

# SQLに慣れている方にとっては、少々くどいかもしれませんが、可読性を高めるために、処理を１つ１つ分解してWITH句で実行しています。
#
# 私がBigQueryの文法に慣れていることもあり、次の3点で修正が必要でした。他にも微妙な差はありそうなので、都度都度直す必要がありますね。
# ・コメントアウトは#は使わず、-- で行う
# ・文字列はダブルコーテーション（ " )ではなく、シングルクオーテーション（ ' )でくくる
# ・日付項目は、文字列を自動で日付型化しないため、DATETIME '2000-11-01'のように日付型を明示的に指定する

# まずは日次で集計してみましょう

df_output = duckdb.query(
    f"""
    WITH t_customer AS(
      -- Frequencyでは、日付（Time）が同じ来店は1来店とみなして集計
      SELECT CustID, max(Time) AS last_purchase_date, COUNT(DISTINCT Time) AS Frequency, SUM(SalesPrice * Amount) AS Monetary
      FROM df
      WHERE Time BETWEEN DATETIME '2001-01-01' AND DATETIME '2001-02-28'
      GROUP BY CustID
    ),
    t_calc_Recentry AS(
      SELECT
        CustId, DATE_DIFF('day', CAST(last_purchase_date AS DATE), CAST('2001-03-01' AS DATE)) AS Recentry, Frequency, Monetary
      FROM t_customer
    )
    SELECT
      CustId, Recentry, Frequency, Monetary,
      NTILE(3) OVER(ORDER BY Recentry) AS R, -- Recentryは値が小さい（最新購入日が最近に近い）ほど良い
      NTILE(3) OVER(ORDER BY Frequency DESC) AS F,
      NTILE(3) OVER(ORDER BY Monetary DESC) AS M
    FROM t_calc_Recentry
    """
).to_df()

In [36]:
df_output.head()

,CustID,Recentry,Frequency,Monetary,R,F,M
0,1622362,12,3,355201708.0,2,1,1
1,1593020,10,6,6591632.0,2,1,1
2,1062489,2,11,5418335.0,1,1,1
3,2123936,8,17,4273159.0,1,1,1
4,2139296,31,2,3076480.0,3,2,1


作成したテーブルのアウトプット
*   CustID：顧客コード。元々あって今回集計キーにした項目
*   Recentry：最新購入日から本日までの経過日数
*   Frequency：期間内の総来店回数
*   Monetary：期間内の総購入金額
*   R：最新購入日に基づいたRの評価
*   F：来店回数に基づいたFの評価
*   M：購入金額に基づいたMの評価

それぞれのランクごとに人数を出してみます。

が、1つの軸だと、同じ人数のため、あまり意味がありません。

In [38]:
df_output["R"].value_counts()

1    8068
2    8067
3    8067
Name: R, dtype: int64

2つの軸をクロス集計することで、見えてくるものがあります。

In [39]:
pd.crosstab(df_output['R'], df_output['F'])

F,1,2,3
R,,,
1,5133,2935,0
2,2455,3820,1792
3,480,1312,6275


Recentry * Frequencyのクロス集計

両方とも同じランクの顧客が多いですが、Rが1なのにFが2の顧客や、Fが1なのにRが2の顧客がいます。

このランクを引き上げる施策（例えばFを上げるのであれば、月〇回来店したら〇％オフなど）を進めると、どの程度購入金額に影響するでしょうか。

In [46]:
# R=1は固定して、Fによってどの程度1人あたりの購入金額が変わるかを集計

df_output_r1 = duckdb.query(
    f"""
    SELECT
    R, F, SUM(Monetary) / COUNT(DISTINCT CustID) AS Average_Monetary
    FROM df_output
    WHERE R = 1
    GROUP BY R, F
    ORDER BY F
    """
).to_df()

In [47]:
 # データ確認
df_output_r1

,R,F,Average_Monetary
0,1,1,11087.074031
1,1,2,2426.578194


上記のように、Fのランクを2から1にあげることで、購入金額として+\8,000弱を期待することができます。

今回はわかりやすく、R, F, Mともに3段階にしました。

これを必要に応じて段階を5や10に増やすことで、どの顧客ランクの層を引き上げると最も売上に貢献しそうかを分析し、具体的なターゲティングが可能になります。